In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split



In [10]:
breast_cancer_data = pd.read_csv('data.txt', header=None)

In [11]:
train, test = train_test_split(breast_cancer_data, test_size=0.2)

In [12]:
train

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
164,8712289,M,23.270,22.04,152.10,1686.0,0.08439,0.11450,0.132400,0.097020,...,28.010,28.22,184.20,2403.0,0.12280,0.35830,0.394800,0.234600,0.3589,0.09187
429,90769602,B,12.720,17.67,80.98,501.3,0.07896,0.04522,0.014020,0.018350,...,13.820,20.96,88.87,586.8,0.10680,0.09605,0.034690,0.036120,0.2165,0.06025
461,911296202,M,27.420,26.27,186.90,2501.0,0.10840,0.19880,0.363500,0.168900,...,36.040,31.37,251.20,4254.0,0.13570,0.42560,0.683300,0.262500,0.2641,0.07427
93,862009,B,13.450,18.30,86.60,555.1,0.10220,0.08165,0.039740,0.027800,...,15.100,25.94,97.59,699.4,0.13390,0.17510,0.138100,0.079110,0.2678,0.06603
116,864726,B,8.950,15.76,58.74,245.2,0.09462,0.12430,0.092630,0.023080,...,9.414,17.07,63.34,270.0,0.11790,0.18790,0.154400,0.038460,0.1652,0.07722
206,879804,B,9.876,17.27,62.92,295.4,0.10890,0.07232,0.017560,0.019520,...,10.420,23.22,67.08,331.6,0.14150,0.12470,0.062130,0.055880,0.2989,0.07380
537,919812,B,11.690,24.44,76.37,406.4,0.12360,0.15520,0.045150,0.045310,...,12.980,32.19,86.12,487.7,0.17680,0.32510,0.139500,0.130800,0.2803,0.09970
378,9013594,B,13.660,15.15,88.27,580.6,0.08268,0.07548,0.042490,0.024710,...,14.540,19.64,97.96,657.0,0.12750,0.31040,0.256900,0.105400,0.3387,0.09638
529,918465,B,12.070,13.44,77.83,445.2,0.11000,0.09009,0.037810,0.027980,...,13.450,15.77,86.92,549.9,0.15210,0.16320,0.162200,0.073930,0.2781,0.08052
54,857438,M,15.100,22.02,97.26,712.8,0.09056,0.07081,0.052530,0.033340,...,18.100,31.69,117.70,1030.0,0.13890,0.20570,0.271200,0.153000,0.2675,0.07873


In [8]:
test

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
513,915940,B,14.580,13.66,94.29,658.8,0.09832,0.08918,0.082220,0.043490,...,16.760,17.24,108.50,862.0,0.12230,0.19280,0.249200,0.09186,0.2626,0.07048
523,917896,B,13.710,18.68,88.73,571.0,0.09916,0.10700,0.053850,0.037830,...,15.110,25.63,99.43,701.9,0.14250,0.25660,0.193500,0.12840,0.2849,0.09031
102,862965,B,12.180,20.52,77.22,458.7,0.08013,0.04038,0.023830,0.017700,...,13.340,32.84,84.58,547.8,0.11230,0.08862,0.114500,0.07431,0.2694,0.06878
363,9010872,B,16.500,18.29,106.60,838.1,0.09686,0.08468,0.058620,0.048350,...,18.130,25.45,117.20,1009.0,0.13380,0.16790,0.166300,0.09123,0.2394,0.06469
93,862009,B,13.450,18.30,86.60,555.1,0.10220,0.08165,0.039740,0.027800,...,15.100,25.94,97.59,699.4,0.13390,0.17510,0.138100,0.07911,0.2678,0.06603
296,891936,B,10.910,12.35,69.14,363.7,0.08518,0.04721,0.012360,0.013690,...,11.370,14.82,72.42,392.2,0.09312,0.07506,0.028840,0.03194,0.2143,0.06643
226,88147101,B,10.440,15.46,66.62,329.6,0.10530,0.07722,0.006643,0.012160,...,11.520,19.80,73.47,395.4,0.13410,0.11530,0.026390,0.04464,0.2615,0.08269
539,921362,B,7.691,25.44,48.34,170.4,0.08668,0.11990,0.092520,0.013640,...,8.678,31.89,54.49,223.6,0.15960,0.30640,0.339300,0.05000,0.2790,0.10660
322,894855,B,12.860,13.32,82.82,504.8,0.11340,0.08834,0.038000,0.034000,...,14.040,21.08,92.80,599.5,0.15470,0.22310,0.179100,0.11550,0.2382,0.08553
525,91805,B,8.571,13.10,54.53,221.3,0.10360,0.07632,0.025650,0.015100,...,9.473,18.45,63.30,275.6,0.16410,0.22350,0.175400,0.08512,0.2983,0.10490
